# Пострение спам-фильтра при помощи алгоритма Наивного байеса

**Цель**: создать спам-фильтр для смс сообщений

**Требования к работе спам-фильтра:** 
1. Узнает, как люди классифицируют сообщения.
2. Использует эти человеческие знания для оценки вероятностей для новых сообщений - вероятности для спама и не спама.
3. Классифицирует новое сообщение на основе этих значений вероятности - если вероятность спама выше, то он классифицирует сообщение как спам. В противном случае он классифицирует его как не спам (если два значения вероятности равны, тогда нам может потребоваться человек для классификации сообщения).

Набор данных был составлен Тиаго А. Алмейдой и Хосе Мария Гомес Идальго, и его можно загрузить из [репозитория машинного обучения UCI](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) .  Процесс сбора данных описан более подробно [на этой странице](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition) , где вы также можете найти некоторые работы авторов.

# Исследование, очистка и предварительная подготовка данных

In [198]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()


(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [199]:
spam_proba=len(sms_spam[sms_spam["Label"]=="spam"])/len(sms_spam)
nonspam_proba=len(sms_spam[sms_spam["Label"]=="ham"])/len(sms_spam)

print("Вероятность получить спам равна {} %, не спам {}%".format(round(spam_proba*100,2),round(nonspam_proba*100,2)))

Вероятность получить спам равна 13.41 %, не спам 86.59%


Тк кол-во не спам сообщений в наборе данных 85,96%, то будем ожидать близкого показателя точности работы алгоритма для тестового набора данных. **Установим критерий качества работы алгоритма 80%**

In [200]:
#Разобъем наши данные на тестовую и обучающие выборки для определения точности работы классификатора. Для этого рандомизируем их.
spam_random=sms_spam.sample(frac=1,random_state=1)

# Рассчитываем индекс для разделения
training_test_index = round(len(spam_random) * 0.8)

# Формируем тренировочный и тестовый набор
training_set = spam_random[:training_test_index].reset_index(drop=True)
test_set = spam_random[training_test_index:].reset_index(drop=True)
print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [201]:
a=round(training_set["Label"].value_counts(normalize=True)[0]*100,2)
b=round(training_set["Label"].value_counts(normalize=True)[1]*100,2)
print("Вероятность получить спам для тренировочного набора данных равна {} %, не спам {}%".format(a,b))

Вероятность получить спам для тренировочного набора данных равна 86.54 %, не спам 13.46%


In [202]:
a=round(test_set["Label"].value_counts(normalize=True)[0]*100,2)
b=round(test_set["Label"].value_counts(normalize=True)[1]*100,2)
print("Вероятность получить спам для тестового набора данных равна {} %, не спам {}%".format(a,b))

Вероятность получить спам для тестового набора данных равна 86.8 %, не спам 13.2%


In [203]:
# Очистка данных
training_set["SMS"]=training_set["SMS"].str.replace(r'[^\w\s]+', '')
training_set["SMS"]=training_set["SMS"].str.lower()

In [204]:
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth theres a card on da ...


In [205]:
vocabulary=[]

In [206]:
for i in training_set["SMS"].str.split():
    for j in i:
        vocabulary.append(j)
vocabulary=list(set(vocabulary))

In [207]:
len(vocabulary)

8448

In [209]:
#Сформирую словарь с кол-м слов в каждом сообщении
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS'].str.split()):
    for word in sms:
        if word not in vocabulary:
            word_counts_per_sms[word][index]=1
        else:
            word_counts_per_sms[word][index]+= 1

#Преобразую словарь в набор данных           
word_counts=pd.DataFrame(word_counts_per_sms)
word_counts.head()

#Объединим полученный набор с метками сообзений из исходного набора
training_set_clean=pd.concat([training_set["Label"], word_counts], axis=1)
training_set_clean.head()

,Label,2wks,real,unspoken,messaged,bollox,62468,spose,workand,franyxxxxx,...,leading,give,culdnt,mall,wining,signal,needing,ujhhhhhhh,lonely,playerwhy
0,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Разработка спам-фильтра

После очистки данных можно приступить к построению спам фильтра.
Для этого нам необходимо рассчитать два значения ниже

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Кроме того, чтобы вычислить P (w i | Spam) и P (w i | Ham) в формулах выше, напомним, что нам нужно использовать эти уравнения:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}\alpha = 1 -сглаживае Лапласса\end{equation}

**NSpam**- равно количеству слов во всех спам-сообщениях — это не количество спам-сообщений, и это не общее количество уникальных слов в спам-сообщениях.

**NHam** равен количеству слов во всех не-спам-сообщениях — это не количество не-спам-сообщений, и это не общее количество уникальных слов в не-спам-сообщениях.

In [144]:
alpha = 1
n_spam=len(training_set_clean[training_set_clean["Label"]=="spam"])
n_ham=len(training_set_clean[training_set_clean["Label"]=="ham"])
n_vocabulary=len(vocabulary)
p_spam = training_set_clean['Label'].value_counts(normalize=True)['spam']
p_ham = training_set_clean['Label'].value_counts(normalize=True)['ham']

print("n_spam={},n_hum={},n_vocabulary={},p_spam={},p_ham={}".format(n_spam,n_hum,n_vocabulary,p_spam,p_ham))

n_spam=600,n_hum=3858,n_vocabulary=8449,p_spam=0.13458950201884254,p_ham=0.8654104979811574


Теперь перейдем к расчету параметров:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [145]:
# Инициализация словарей для записи значений параметров
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Выделим в отдельные наборы спам и не спам сообщения
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

In [146]:
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

In [148]:
import re

def classify(message):
"""Функция для классификации сообщений"""
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [149]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 5.421049348813775e-22
P(Ham|message): 5.441579473429447e-21
Label: Ham


In [150]:
classify('Sounds good, Tom, then see u there')

P(Spam|message): 1.0012419359896888e-22
P(Ham|message): 2.886044557258971e-16
Label: Ham


# Расчет точности спам-фильтра

In [211]:
# Изменим вывод функции с печати на возврат значения для рассчета точности работы классификатора

def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'Equal proabilities, have a human classify this!'

In [212]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Точность рассчитаю по формуле:
\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

In [215]:
def accuracy(data):
    """Рассчет точности работы классификатора для размеченного им набора данных"""
    correct=0
    total=len(test_set)
    for n,i in data.iterrows():
        if i["Label"]==i["predicted"]:
            correct+=1
    return round(correct/total*100,2), correct,total-correct
        
    
    

In [216]:
a,b,c=accuracy(test_set)
print(a,b,c)

95.96 1069 45


**Вывод**:в этом проекте построен спам-фильтр для SMS-сообщений, используя мультиномиальный наивный алгоритм Байеса. Фильтр имел точность 95,96% на тестовом наборе, что можно считать превосходным результатом. Мы изначально стремились к точности не менее 80%, но улалось добиться лучшего результата.